# TP General Physics / M1

## High Velocity Cloud Analysis in HI4PI Data


##### author: Louis Legrand, adapted from Antoine Marchal, 2019/2020



## 1. Introduction 

The study of our own galaxy, the Milky Way, is complicated by the fact that we are within it.
The sun, located at 8.5 kpc from the galactic center, moves in a circular orbit with a radial velocity of
220 km.s$^{-1}$.
As a first approximation, we can discribe the galactic plane as a sum of three components : Gas (molecular and neutral hydrogen), Star
and Dark Matter. Around this galactic plane, in what is called the galactic halo, there is an important gas reservoir mainly composed of neutral gas (HI).
It turns out that this reservoir could be an important source of gas for the formation of molecular clouds, the place where stars are formed.
To better understand the content and nature of this reservoir, we propose in this work to study one of these clouds, called High Velocity Clouds (HVC).




<figure>
  <img src="img/simulation.png" alt="simulation.png" style="width: 60%;"/>
  <figcaption>Figure 1: (a) Integrated intensity, (b) centroid velocity, and (c) velocity dispersion of a model CHVC (model Wb1a15b of
    Heitsch & Putman 2009), traveling at $45\,^{\circ}$ to the observer. Contours are given at [1, 5, 9, 13] K km s-1.
    (from Heitsch et al, 2016)</figcaption>
</figure>


Since their discorevy by Muller et al (1963), High Velocity Clouds are studied as isolated objects. They are defined
as HI clouds with a radial velocity (typically $ V_r \approx 200 \, km.s^{-1}$) that cannot be explained by the rotation
curve of the Milky Way (Wakker 1991). Figure 1 shows an HVC view from a numerical simulation performed by
Heitsch et al, 2016. HI is observed since many years through the 21 cm hyperfine structure line, and the latest full sky survey
has been released by the HI4PI collaboration. More details are present in this recent article :
https://arxiv.org/abs/1610.06175.
The research project we adress here is based on a single HVC cloud named HVC125+41-207.
Starting from a part of this survey, we develop a simple method to obtain some physical properties of this cloud
and its environment.

Some useful constants:
- Atomic mass of hydrogen : $m_H = 1.6737236 \times 10^{-27} \, kg$
- Definition of a parsec : $pc = 3.085677581467192 \times 10^{16} \, m$
- Mass of the Sun : $M_{\odot} = 1.9891 \times 10^{30} \, kg$
- Mean mass per particle within the sphere : $\mu \approx 1.25 \, m_H$
- Boltzmann constant : $k = 1.38064852 \times 10^{-23} \, m^2.kg.s^{-2}.K^{-1}$


## 2. Getting started - hyperspectral cube

### 2.1 Reading Data from FITS Files

This work is based on an hyperspectral observation of the 21 cm line
(see representation figure 2). For each line of sigh across the observation, a simple Doppler effet calculation allows us to obtain a projected velocity spectrum. 

<figure>
  <img src="img/hyperspectral.png" alt="hyperspectral" style="width: 40%;"/>
  <figcaption>Figure 2: 3D representation of a hyperspectral cube. All the points on a straight line in this cube compose a spectrum.
  We can select a sub-regoin of this cube which corresponds to a smaller part of the sky.</figcaption>
</figure>

We propose to read this hyperspectral observation formatted in FITS using python.
A good documentation is available via the following link: http://docs.astropy.org/en/stable/io/fits/

First download the data cube. In your terminal go to the folder where this notebook is saved and run:

```batch
wget http://cdsarc.u-strasbg.fr/ftp/J/A+A/594/A116/CUBES/GAL/CAR/CAR_G07.fits```



Some useful librairies to import.   
(Press SHIFT+ENTER inside a cell to run it.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs

We load the hyperspectral cude using astropy.io.
The hdulist is a list containing HDU objects (in our case there is only one hdu object). 
Each HDU object has a header and a data attributes. The header contains all the relevant information to exploit the data.

In [ ]:
hdu_list= fits.open("CAR_G07.fits")
hdu = hdu_list[0]

header = hdu.header
data = hdu.data

In [ ]:
data.shape

We can see that our data cube has three dimensions.
The first dimension has a lenght of 933, the other two have a lenght of 266.

Can you say to which physical measure each dimenson correspond ?

### 2.2 Ploting data 

Each pixel in this cube is a brightness temperature, given in Kelvin. 
We can play a bit with this cube, for instance we can plot a 2D slice.

In [ ]:
plt.figure()
plt.imshow(data[450], origin='lower')
plt.colorbar(label='$T_b$ [K]')

As you can see, the plot above has no legends on the axes, we need to convert the index of pixels into physical values. This information is contained in the header.

In [ ]:
header

We see that the header object contains several fields. 
To associate a physicalquantity to the pixel counts we will use the CRVAL, CRPIX and CDELT.

CRVAL is the physical value (in unit CUNIT) associated to the pixel number CRPIX. CDELT is the increment between to pixels. 

Write a function that returns the velocity corresponding to the pixel index pix.

In [ ]:
CRPIX = 0  # complete here 
CRVAL = 0
CDELT = 0

def pix2vel(pix, CRPIX, CRVAL, CDELT):
    """Returns the velocity (in km/s) correpsonding to the pixel pix."""
    vel = 0  #Complete here 
    return vel  

Now we can plot a spectrum for one given position in the sky.

In [ ]:
# Plot a velocity spectrum for one pixel in the sky 

To plot the correct coordinates in the galactic referential, we will use a wcs object, which easily deals with this.

In [ ]:
hvc_wcs = wcs.WCS(header)
hvc_wcs2d = hvc_wcs.dropaxis(2)

plt.figure()

# Create a plot in the correct angular frame
plt.subplot(projection=hvc_wcs2d)

# Exemple of how to plot a slice of the cube in the galactic frame. 
plt.imshow(data[0])

## 2.3 High Velocity Cloud location

From the Temperature brightness as a function of velocity, we can compute the column density (in number of Hydrogen atoms per surface) :

$$ 
\frac{ N_{HI}}{cm^{-2}} = 1.82243 \times 10^{18} \times \int_{-\infty}^{+\infty} \left(\frac{T_b(v)}{K} \right) d\left( \frac{v}{km.s^{-1}}\right)
$$


Note that we can direclty obtain the surface density :

$$
\begin{align}
  \Sigma = \frac{\left<N_{HI}\right> \, m_H \, pc^2}{M_\odot}
\end{align}
$$
where $m_H$ is the atomic mass of hydrogen, $pc$ is the definition of a parsec,
and $M_{\odot}$ is the mass of the Sun.


A first exercice would be to visualize the total integreted column density map of the field. 

In [ ]:
### Compute the integrated column density map and plot it.


There is no clear evidence of an HVC in this map, simply because the total integrated density is dominated
by the HI emission of the Milky Way. 
Indeed, the HVC HVC125+41-207 is located in the following spectral range :

$v_{rad} (km.s^{-1}) = [-225, -185]$.

From the article describing the HI4PI survey, it is also possible to clean the cube from the instrumental noise. 
For exemple we can keep all values greater than $3 \times \sigma_{rms}$, with $\sigma_{rms}$ the sensitivity of the radio telescope used.

After clenaing the cuba and selecting the spectral range, we can plot the HVC.

Note that the galactic longitude range and the galactic latitude range of the cube are respectively
$l(deg) \in [119, 141]$ and $b(deg) \in [19, 51]$. We can zoom in on this rectangle of the sky to have a celar vview of the HVC.

In [ ]:
### Plot the column density map of the HVC.

## 3. Physical properties of the HVC

### 3.1 Mean spectra of the HVC


To obtain some physical properties of this clump, we can analize the mean spectrum of the HVC. Figure 3.1 shows a schematic view of the numerical calculation required to obtain the average spectrum of the cloud from the hyperspectral cube.


<figure>
  <img src="img/spectra.png" alt="hyperspectral" style="width: 60%;"/>
  <figcaption>Figure 3: Representation of the passage of hyperspectral data to the mean spectrum of the cloud.</figcaption>
</figure>


Considering that the observed velocity dispersion is only due to the thermal Doppler broadening, the kinetic temperature
of our system $T_k$ can be written as :
$$
\begin{align}
  T_k = \frac{m_H \, \sigma_v^2}{k} \, (K),
\end{align}
$$
where $\sigma_v$ is computed by fitting a Gaussian function on the average spectrum obtained previously.


In [ ]:
### Compute the mean temperature of the cloud

### 3.2 Two phases in the cloud 

Following the work of Bruns et al. 2001, and as we can see on the velocity spectrum, it seems that there are two phases.

In [ ]:
### Find the temperature of the two phases.

In [ ]:
### Show the location of the two phases 

### 3.3 Pressure in the cloud

As a first approximation, we can applied the Virial equilibrium to obtain the pressure outside the HVC as a function
of the distance d (preferentialy expressed in kpc which is a unit of length used in astronomy).
It is nothing more than the balance between the inner and outer medium.

$$
\begin{align}
  \frac{P_s}{k} = \underbrace{\frac{\left< N_{HI} \right> \, T_k}{d \, \theta}}_\text{kinetic pressure} \,
  \underbrace{- \, \frac{\mu^2 \, G \, \pi \, \left< N_{HI} \right>^2}{15 \, k}}_\text{gravitational pressure}
\end{align}
$$

where $\theta$ is the observed angular diameter in radian, $T_k$ is the kinetic temperature of the gas, d is the distance of the clump,
$G$ and $k$ and respectively the gravitational constant and the Boltzmann constant, $\left< N_{HI} \right>$ is the mean value
of the column density and $\mu$ the mean mass per particle within the sphere.


In [ ]:
### Compute the pressure inside the cloud, as a function of distance

### 3.4 Compute the distance to the cloud

Using equation 7 from Wolfire et al. 1995, one can compute the distance to the cloud.

In [ ]:
### Compute the distance to the cloud

### 3.5 Get the mass of the cloud

Using computations above, one can compute the mass of the cloud.